In [1]:
%pylab inline
from astropy.table import Table, hstack,vstack, Column, QTable
from astropy.io import fits
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import FK5
from astropy.time import Time
from astropy.coordinates import Angle
from astropy.io import ascii
import numpy.ma as ma

Populating the interactive namespace from numpy and matplotlib


In [2]:
tsc = Table.read('/Users/JanviMadhani/thaw_star_catalog/finaltable.csv')

In [3]:
### Add in 1950 coordinates columns
RA1950 = Column(np.arange(len(tsc)), name='RA1950')
tsc.add_column(RA1950, index=9)
tsc['RA1950'] = tsc['RA1950'].astype(str)

Dec1950 = Column(np.arange(len(tsc)), name='Dec1950')
tsc.add_column(Dec1950, index=10)
tsc['Dec1950'] = tsc['Dec1950'].astype(str)

## Precess to 1950 Coordinates using YPC 1900 Coordinates

In [4]:
def convert_to_J1950(ra,dec,start_equinox='J1900'):
    """
    Give function RA hour, min, seconds,
                  Dec deg, min, seconds
                  Starting equinox: default = 1900
    Get RA in format: 00h00m00.00s
    Dec in format: 00d00m00.00s
    """
        
    if ra:
        
        fk0c = SkyCoord(ra, dec, unit=(u.hourangle, u.deg), frame=FK5(equinox=Time(start_equinox)))
        fk0_1950 = FK5(equinox='J1950')
        c = fk0c.transform_to(fk0_1950,type)
        string = c.to_string('hmsdms',sep=(' ',' '))
        ra1950 = string[:12]
        dec1950 = string[14:]
        
        
    else:
        ra1950,dec1950 = 0,0
        c = 0 

    
    return(ra1950,dec1950)

In [5]:
ra = '00 13 0.00'
dec = '+36 14 0.00'
convert_to_J1950(ra,dec)

('00 15 36.740', '+36 30 40.4597')

In [6]:
for i in range(len(tsc)):
    ra = tsc['YPC RA1900'][i]
    dec = tsc['YPC Dec1900'][i]
    ra1950,dec1950 = convert_to_J1950(ra,dec)
    tsc['RA1950'][i] = ra1950
    tsc['Dec1950'][i] = dec1950

In [7]:
tsc.colnames

['Thaw Number',
 'YPC Number',
 'Thaw Name',
 'YPC Name',
 'BD Number',
 'Thaw RA1900',
 'Thaw Dec1900',
 'YPC RA1900',
 'YPC Dec1900',
 'RA1950',
 'Dec1950',
 'RA2000',
 'Dec2000',
 'Thaw B',
 'YPC (B-V)',
 'YPC V',
 'Thaw Spectrum',
 'YPC Spectrum',
 'Thaw Rel. Parallax',
 'Thaw Rel. Parallax Error',
 'Delta Pi',
 'Thaw Abs. Parallax',
 'Yale Abs. Parallax',
 'Yale Abs. Parallax Error',
 'Yale Tot. PM',
 'Yale PM Position Angle',
 'No. of Thaw Observations',
 'Thaw Footnotes']

In [8]:
tsc['Yale Abs. Parallax'] = tsc['Yale Abs. Parallax'].astype(str)
tsc['Yale Abs. Parallax Error'] = tsc['Yale Abs. Parallax Error'].astype(str)
tsc['YPC Spectrum'] = tsc['YPC Spectrum'].astype(str)
tsc['YPC V'] = tsc['YPC V'].astype(str)
tsc['YPC (B-V)'] = tsc['YPC (B-V)'].astype(str)

for i in range(len(tsc)):
    s1 = tsc['YPC Spectrum'][i]
    s1 = (str(s1.astype(str)))
    tsc['YPC Spectrum'][i] = s1

    s2 = tsc['YPC V'][i]
    s2 = (str(s2.astype(str)))    
    tsc['YPC V'][i] = s2 
    
    s3 = tsc['YPC (B-V)'][i]
    s3 = (str(s3.astype(str)))  
    tsc['YPC (B-V)'][i] = s3

    

In [9]:
tsc.colnames

['Thaw Number',
 'YPC Number',
 'Thaw Name',
 'YPC Name',
 'BD Number',
 'Thaw RA1900',
 'Thaw Dec1900',
 'YPC RA1900',
 'YPC Dec1900',
 'RA1950',
 'Dec1950',
 'RA2000',
 'Dec2000',
 'Thaw B',
 'YPC (B-V)',
 'YPC V',
 'Thaw Spectrum',
 'YPC Spectrum',
 'Thaw Rel. Parallax',
 'Thaw Rel. Parallax Error',
 'Delta Pi',
 'Thaw Abs. Parallax',
 'Yale Abs. Parallax',
 'Yale Abs. Parallax Error',
 'Yale Tot. PM',
 'Yale PM Position Angle',
 'No. of Thaw Observations',
 'Thaw Footnotes']

In [10]:
tsc['YPC Number'] = tsc['YPC Number'].astype(str)
tsc['YPC Name'] = tsc['YPC Name'].astype(str)
tsc['YPC RA1900'] = tsc['YPC RA1900'].astype(str)
tsc['YPC Dec1900'] = tsc['YPC Dec1900'].astype(str)
tsc['YPC (B-V)'] = tsc['YPC (B-V)'].astype(str)
tsc['YPC V'] = tsc['YPC V'].astype(str)
tsc['YPC Spectrum'] = tsc['YPC Spectrum'].astype(str)
tsc['Yale Abs. Parallax'] = tsc['Yale Abs. Parallax'].astype(str)
tsc['Yale Abs. Parallax Error'] = tsc['Yale Abs. Parallax Error'].astype(str)
tsc['Yale Tot. PM'] = tsc['Yale Tot. PM'].astype(str)



#tsc.show_in_notebook()

In [11]:
tsc.show_in_notebook()

idx,Thaw Number,YPC Number,Thaw Name,YPC Name,BD Number,Thaw RA1900,Thaw Dec1900,YPC RA1900,YPC Dec1900,RA1950,Dec1950,RA2000,Dec2000,Thaw B,YPC (B-V),YPC V,Thaw Spectrum,YPC Spectrum,Thaw Rel. Parallax,Thaw Rel. Parallax Error,Delta Pi,Thaw Abs. Parallax,Yale Abs. Parallax,Yale Abs. Parallax Error,Yale Tot. PM,Yale PM Position Angle,No. of Thaw Observations,Thaw Footnotes
0,1,51.00,Sigma Andromedae,SIG And,+35 44,00 13 0.00,+36 14 0.00,00 13 06.0,+36 13 51,00 15 42.761,+36 30 31.4322,00 18 19.7,+36 47 07,4.5,4.52,0.05,A2,A2 V,12,8,--,--,19,12.5,0.074,240,--,--
1,2,158.00,Delta Piscium,DEL Psc,+6 107,00 43 0.00,+7 02 0.00,00 43 29.5,+07 02 27,00 46 04.803,+07 18 50.3317,00 48 40.9,+07 35 06,4.5,4.43,1.5,K5,K4 IIIb,12,7,--,--,17,9.4,0.095,119,--,--
2,3,209.00,Epsilon Piscium,EPS Psc,+7 153,00 58 0.00,+7 21 0.00,00 57 45.0,+07 21 06,01 00 20.907,+07 37 15.3213,01 02 56.5,+07 53 24,4.5,4.28,0.96,G5,K0 III,23,8,--,--,26,6.5,0.085,289,--,--
3,4,219.00A,Mu Cassiopeiae,MU Cas,+54 223,01 02 0.00,+54 26 0.00,01 01 36.7,+54 25 47,01 04 35.902,+54 41 51.6498,01 08 16.2,+54 55 11,5.3,5.17,0.69,G5,G5 Vb,143,9,--,--,133,1.7,3.77,115,--,--
4,5,368.00,Omicron Piscium,OMI Psc,+8 273,01 40 0.00,+8 39 0.00,01 40 06.6,+08 39 16,01 42 44.689,+08 54 21.8334,01 45 23.6,+09 09 28,4.5,4.26,0.96,G5,G8 III,12,6,--,--,21,7.7,0.089,53,--,--
5,6,394.00,Beta Arietis,BET Ari,+20 306,01 49 0.00,+20 19 0.00,01 49 06.8,+20 19 09,01 51 52.018,+20 33 57.1538,01 54 38.4,+20 48 28,2.7,2.64,0.13,A5,A5 V,72,8,--,--,67,7.8,0.144,138,--,--
6,7,567.00,39 Arietis,39 Ari,+28 462,02 42 0.00,+28 50 0.00,02 41 57.0,+28 49 55,02 44 54.840,+29 02 33.1509,02 47 54.5,+29 14 49,4.6,4.51,1.11,K,K1.5 III,22,6,--,--,20,1.9,0.207,130,--,--
7,8,897.00A,37 Tauri,37 Tau,+21 585,03 59 0.00,+21 49 0.00,03 58 46.8,+21 48 31,04 01 43.724,+21 56 51.2328,04 04 41.7,+22 04 54,4.5,4.36,1.07,K,K0+ III-*,-22,9,--,--,12,5.7,0.107,122,--,--
8,--,897.00B,--,--,--,--,--,03 58 45.3,+21 46 16,04 01 42.178,+21 54 36.329,04 04 39.4,+22 02 43,--,10.4,--,--,G,--,--,--,--,--,--,0.02,172,--,--
9,9,911.00,50 Persei,50 Per,+37 882,04 02 0.00,+37 47 0.00,04 01 56.5,+37 46 41,04 05 15.538,+37 54 48.4982,04 08 36.6,+38 02 23,5.6,5.51,0.46,F,F7 V,43,8,--,--,47,12.5,0.261,140,--,--


## Create Subtables

In [12]:
#define colnames

colnames = ['TPC No', 'AO Result Published', 'AO Rel. Parallax (err)', 'AO Abs. Parallax (err)', 'YPC No', 'YPC Abs. Parallax (err)', 
           'AO Name', 'YPC Name', 'BD Name', 'RA2000','Dec2000','AO RA1900','AO Dec1900','YPC Total Proper Motion', 'YPC Proper Motion Position Angle','YPC Spectrum',
           'YPC V Mag', 'YPC B-V Mag','AO Obs No, Quality, Date']
vals = [i for i in range(len(colnames))]
for i in range(len(tsc)):
    if tsc[i]['Thaw RA1900']:
        print(i)
        subtable = Table(names=colnames,masked = True,dtype = [object,object,object,object,object,object,object,object,object,object,
                                                object,object,object,object,object,object,object,object,object])
        tpc_no = tsc[i]['Thaw Number']
        
        ao_pub_date = 'yyyymmdd'
        
        if tsc[i]['Thaw Rel. Parallax']:
            ao_rel_p = tsc[i]['Thaw Rel. Parallax'] + str(' (') + tsc[i]['Thaw Rel. Parallax Error'] + str(')')
        else:
            ao_rel_p = ""
        
        ao_abs_p = ''
        ypc_no = str(tsc[i]['YPC Number'])
        ypc_abs_p = str(tsc[i]['Yale Abs. Parallax']) + str(" (") + str(tsc[i]['Yale Abs. Parallax Error']) + str(")")
        ao_name = tsc[i]['Thaw Name']
        ypc_name = str(tsc[i]['YPC Name'])
        bd = tsc[i]['BD Number']
        ra2000 = tsc[i]['RA2000']
        dec2000 = tsc[i]['Dec2000']
        ra1900 = tsc[i]['Thaw RA1900']
        dec1900 = tsc[i]['Thaw Dec1900']
        pm = str(tsc[i]['Yale Tot. PM'])
        pmPA = str(tsc[i]['Yale PM Position Angle'])
        
        
        subtable['YPC Spectrum'] = []
        subtable['YPC Spectrum'] = subtable['YPC Spectrum'].astype(object)
        ypc_spec = str(tsc['YPC Spectrum'][i])
        
        
  
        subtable['YPC V Mag'] = []
        subtable['YPC V Mag'] = subtable['YPC V Mag'].astype(object)
        ypc_v = str(tsc['YPC V'][i])
        
        
        subtable['YPC B-V Mag'] = []
        subtable['YPC B-V Mag'] = subtable['YPC B-V Mag'].astype(object)
        ypc_bv = str(tsc['YPC (B-V)'][i])       
        
        observations = []
        observations.append('xxxxxx q yyyymmdd')
        
        observations = tuple(observations)
        
        subtable.add_row((tpc_no,ao_pub_date,ao_rel_p,ao_abs_p,ypc_no,ypc_abs_p,ao_name,ypc_name,bd,
                         ra2000,dec2000,ra1900,dec1900,pm,pmPA,ypc_spec,ypc_v,ypc_bv,observations))
        
        tpc_no = str(tpc_no)
        cat_no = tpc_no.zfill(4)
        filename = 'AO_'+ str(cat_no) + '_subtable.csv'
        
        print(subtable)
        
        ascii.write(subtable, filename, format='csv',fast_writer = False,fill_values=[(ascii.masked, 'N/A')],overwrite=True) 
        
    
    

0
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
     1            yyyymmdd ...        4.52   ('xxxxxx q yyyymmdd',)
1
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
     2            yyyymmdd ...        4.43   ('xxxxxx q yyyymmdd',)
2
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
     3            yyyymmdd ...        4.28   ('xxxxxx q yyyymmdd',)
3
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
     4            yyyymmdd ...        5.17   ('xxxxxx q yyyymmdd',)
4
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
     5            yyyymmdd ...        

54
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
    46            yyyymmdd ...        3.77   ('xxxxxx q yyyymmdd',)
55
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
    47            yyyymmdd ...        2.94   ('xxxxxx q yyyymmdd',)
57
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
    48            yyyymmdd ...         5.1   ('xxxxxx q yyyymmdd',)
58
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
    49            yyyymmdd ...          --   ('xxxxxx q yyyymmdd',)
59
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
    50            yyyymmdd ...   

106
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
    90            yyyymmdd ...        4.36   ('xxxxxx q yyyymmdd',)
107
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
    91            yyyymmdd ...        3.53   ('xxxxxx q yyyymmdd',)
108
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
    92            yyyymmdd ...        3.97   ('xxxxxx q yyyymmdd',)
109
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
    93            yyyymmdd ...        4.16   ('xxxxxx q yyyymmdd',)
110
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
    94            yyyymmdd .

176
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
   138            yyyymmdd ...        11.0   ('xxxxxx q yyyymmdd',)
180
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
   139            yyyymmdd ...        4.35   ('xxxxxx q yyyymmdd',)
181
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
   140            yyyymmdd ...        5.25   ('xxxxxx q yyyymmdd',)
182
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
   141            yyyymmdd ...        6.54   ('xxxxxx q yyyymmdd',)
183
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
   142            yyyymmdd .

236
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
   185            yyyymmdd ...        5.03   ('xxxxxx q yyyymmdd',)
237
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
   186            yyyymmdd ...        2.27   ('xxxxxx q yyyymmdd',)
238
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
   187            yyyymmdd ...        3.44   ('xxxxxx q yyyymmdd',)
240
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
   188            yyyymmdd ...        2.53   ('xxxxxx q yyyymmdd',)
241
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
   189            yyyymmdd .

...

2322
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1851            yyyymmdd ...        6.31   ('xxxxxx q yyyymmdd',)
2323
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1852            yyyymmdd ...        5.39   ('xxxxxx q yyyymmdd',)
2324
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1853            yyyymmdd ...        8.36   ('xxxxxx q yyyymmdd',)
2325
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1854            yyyymmdd ...        5.55   ('xxxxxx q yyyymmdd',)
2326
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1855            yyyym

2380
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1900            yyyymmdd ...        8.38   ('xxxxxx q yyyymmdd',)
2381
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1901            yyyymmdd ...        6.49   ('xxxxxx q yyyymmdd',)
2382
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1902            yyyymmdd ...         6.5   ('xxxxxx q yyyymmdd',)
2383
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1903            yyyymmdd ...        6.48   ('xxxxxx q yyyymmdd',)
2384
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1904            yyyym

2442
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1948            yyyymmdd ...        3.14   ('xxxxxx q yyyymmdd',)
2443
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1949            yyyymmdd ...        2.08   ('xxxxxx q yyyymmdd',)
2444
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1950            yyyymmdd ...         4.8   ('xxxxxx q yyyymmdd',)
2445
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1951            yyyymmdd ...        2.77   ('xxxxxx q yyyymmdd',)
2446
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1952            yyyym

  1994            yyyymmdd ...        11.0   ('xxxxxx q yyyymmdd',)
2495
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1995            yyyymmdd ...        4.76   ('xxxxxx q yyyymmdd',)
2496
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1996            yyyymmdd ...        4.83   ('xxxxxx q yyyymmdd',)
2497
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1997            yyyymmdd ...          --   ('xxxxxx q yyyymmdd',)
2498
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  1998            yyyymmdd ...        4.48   ('xxxxxx q yyyymmdd',)
2499
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ----------------

2552
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  2042            yyyymmdd ...        5.05   ('xxxxxx q yyyymmdd',)
2553
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  2043            yyyymmdd ...        5.06   ('xxxxxx q yyyymmdd',)
2554
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  2044            yyyymmdd ...       11.67   ('xxxxxx q yyyymmdd',)
2556
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  2045            yyyymmdd ...       11.67   ('xxxxxx q yyyymmdd',)
2559
TPC No AO Result Published ... YPC B-V Mag AO Obs No, Quality, Date
------ ------------------- ... ----------- ------------------------
  2046            yyyym

In [13]:
tsc[48]

Thaw Number,YPC Number,Thaw Name,YPC Name,BD Number,Thaw RA1900,Thaw Dec1900,YPC RA1900,YPC Dec1900,RA1950,Dec1950,RA2000,Dec2000,Thaw B,YPC (B-V),YPC V,Thaw Spectrum,YPC Spectrum,Thaw Rel. Parallax,Thaw Rel. Parallax Error,Delta Pi,Thaw Abs. Parallax,Yale Abs. Parallax,Yale Abs. Parallax Error,Yale Tot. PM,Yale PM Position Angle,No. of Thaw Observations,Thaw Footnotes
str5,str8,str32,str11,str10,str11,str11,str10,str9,str21,str21,str10,str9,str12,str32,str32,str9,str10,str6,str6,int64,int64,str21,str32,str32,int64,int64,int64
--,--,Mean,--,--,--,--,--,--,0,0,--,--,--,--,--,--,--,32,7,--,--,--,--,--,--,--,--


## Format Francis' Table

In [14]:
with open('/Users/JanviMadhani/thaw_star_catalog/TSC.txt', 'r+') as data:
    lines = data.readlines()

In [15]:
check1 = '999999 x 99999999 9 9\n'
check2 = '999999 x 99999999 9\n'

match_entries = []
match_entries.append(lines[1])

for i in range(len(lines)):
    if lines[i] == check2:
        match_entries.append(lines[i+1]) 
        
    elif lines[i] == check2:
        match_entries.append(lines[i+1])
        

In [16]:
print(lines[0])
test_split = match_entries[1].split()
print(test_split)
print(len(test_split))

For each Thaw star: the first line is “Field_Name” “Magnitude” “Coordinates (in changing formats)” “Equinox_of_Coordinates”; next lines are “Observation_Plate” “Quality” “Observation_Epoch”; last line is “999999 x 99999999 9 9 9” 

['Hu_60', '8.5', '00', '01', '_', '+39.5', '_', '_', '_']
9


### Initialize astropy table to fill match values

In [39]:
fcolnames = ['Field Name', 'Magnitude', 'RA', 'Dec', 'Equinox','TSC No.','TSC Name','RA1950','Dec1950']
francis_table = Table(names = fcolnames,dtype = [object,object,object,object,object,object,object,object,object])

need_to_check = []

for i in range(len(match_entries)):
    if len(match_entries[i].split()) == 9:
        dat = match_entries[i].split()
        fname = dat[0]
        mag = dat[1]
        
        rah = dat[2]
        if len(rah) < 2:
            rah = '0' + rah
        ram = dat[3]
        ras = dat[4]
        ra = rah + ' ' + ram + ' ' + ras
        
        dec_d = dat[5]
        decm = dat[6]
        decs = dat[7]
        
        if (len(dat[8].split(',')) <=1) & (len(dec_d.split('.')) > 1):
            declination = dec_d.split('.')
            decd = declination[0]
            mfrac = '.' + declination[1]
            minutes = 60 *float(mfrac)
            mins = str(minutes).split('.')
            decm = mins[0]
            secfrac = '.' + mins[1]
            seconds = 60 *float(secfrac)
            s = str(int(seconds))
            if len(s)<2:
                decs = '0' + s
            else:
                decs = s
            
        else:
            decd = dec_d
        
        dec = decd + ' ' + decm + ' ' + decs
        equinox = dat[8]
        tscno = ''
        tscname = ''
        ra1950 = ''
        dec1950 = ''
        
        if (len(dat[8].split(',')) >1):
            need_to_check.append(i)
        
    else:
        need_to_check.append(i)
        fname = ''
        mag = ''
        ra = ''
        dec = ''
        equinox = ''
        tscno = ''
        tscname = ''
        ra1950 = ''
        dec1950 = ''
        
    francis_table.add_row((fname,mag,ra,dec,equinox,tscno,tscname,ra1950,dec1950))

In [40]:
testing = '+23.5'
decli = testing.split('.')
num = '.'+ decli[1]
minute = 60*float(num)
print(str(minute).split('.'))

['30', '0']


In [41]:
francis_table.show_in_notebook()

idx,Field Name,Magnitude,RA,Dec,Equinox,TSC No.,TSC Name,RA1950,Dec1950
0,Lal_47148,7.7,00 00 14,+01 51 _,1950,,,,
1,Hu_60,8.5,00 01 _,+39 30 00,_,,,,
2,3_Ceti,5.2,00 02 _,-10 52 _,_,,,,
3,Sigma_3062,6.9,00 03 38,+58 09 _,1950,,,,
4,Sigma_3060,8.5,00 03 22,+17 48 _,1950,,,,
5,Grb_4220,5.8,00 02 05,+66 53 _,1950,,,,
6,Ambr_7793,6.4,00 02 16,+34 23 _,1950,,,,
7,Sigma_3057,7.2,00 02 19,58 15 13,1950,,,,
8,Porter_1,8.2,00 02 47,+08 31 _,1950,,,,
9,33_Tiscium,4.7,00 03 _,-6 00 _,_,,,,


In [42]:
print(need_to_check)

[27, 29, 93, 122, 148, 152, 179, 183, 188, 198, 244, 248, 259, 323, 359, 366, 391, 422, 441, 636, 795, 836, 879, 932, 938, 1001, 1023, 1143, 1148, 1199, 1202, 1207, 1215, 1290, 1325, 1366, 1401, 1409, 1445, 1463, 1520, 1622, 1833, 1874, 2030]


In [30]:
print(match_entries[27])

beta_1027 7.5 00 13 21.4 _ _ _

